# Pretraining on WT103

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
#export
from exp.nb_12a import *

## Data

One time donwload

In [ ]:
path = datasets.Config().data_path()
version = '103' #2

In [ ]:
! wget https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-{version}-v1.zip -P {path}
! unzip -q -n {path}/wikitext-{version}-v1.zip  -d {path}
! mv {path}/wikitext-{version}/wiki.train.tokens {path}/wikitext-{version}/train.txt
! mv {path}/wikitext-{version}/wiki.valid.tokens {path}/wikitext-{version}/valid.txt
! mv {path}/wikitext-{version}/wiki.test.tokens {path}/wikitext-{version}/test.txt

--2019-04-16 20:58:15--  https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-v1.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.86.173
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.86.173|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190229076 (181M) [application/zip]
Saving to: ‘/home/ubuntu/.fastai/data/wikitext-103-v1.zip’

wikitext-103-v1.zip 100%[===================>] 181.42M  19.0MB/s    in 10s     

2019-04-16 20:58:27 (17.9 MB/s) - ‘/home/ubuntu/.fastai/data/wikitext-103-v1.zip’ saved [190229076/190229076]



Split the articles

In [ ]:
path = datasets.Config().data_path()/'wikitext-103'

In [ ]:
def istitle(line):
    return len(re.findall(r'^ = [^=]* = $', line)) != 0

In [ ]:
def read_wiki(filename):
    articles = []
    with open(filename, encoding='utf8') as f:
        lines = f.readlines()
    current_article = ''
    for i,line in enumerate(lines):
        current_article += line
        if i < len(lines)-2 and lines[i+1] == ' \n' and istitle(lines[i+2]):
            current_article = current_article.replace('<unk>', UNK)
            articles.append(current_article)
            current_article = ''
    current_article = current_article.replace('<unk>', UNK)
    articles.append(current_article)
    return articles

In [ ]:
train = TextList(read_wiki(path/'train.txt'), path=path) #+read_file(path/'test.txt')
valid = TextList(read_wiki(path/'valid.txt'), path=path)

In [ ]:
len(train), len(valid)

(28476, 60)

In [ ]:
sd = SplitData(train, valid)

In [ ]:
proc_tok,proc_num = TokenizeProcessor(),NumericalizeProcessor()

In [ ]:
ll = label_by_func(sd, lambda x: 0, proc_x = [proc_tok,proc_num])

In [ ]:
pickle.dump(ll, open(path/'ld.pkl', 'wb'))

In [ ]:
ll = pickle.load( open(path/'ld.pkl', 'rb'))

In [ ]:
bs,bptt = 128,70
data = lm_databunchify(ll, bs, bptt)

In [ ]:
vocab = ll.train.proc_x[-1].vocab
len(vocab)

60002

## Model

In [ ]:
dps = np.array([0.25, 0.1, 0.2, 0.02, 0.15]) * 0.2

In [ ]:
emb_sz, nh, nl = 300, 300, 2
model = get_language_model(len(vocab), emb_sz, nh, nl, 0, input_p=dps[0], output_p=dps[1], weight_p=dps[2], 
                           embed_p=dps[3], hidden_p=dps[4])

In [ ]:
cbs = [partial(AvgStatsCallback,accuracy_flat),
       CudaCallback, Recorder,
       partial(GradientClipping, clip=0.1),
       partial(RNNTrainer, alpha=2., beta=1.),
       ProgressCallback]

In [ ]:
learn = Learner(model, data, cross_entropy_flat, lr=5e-3, cb_funcs=cbs, opt_func=adam_opt())

In [ ]:
lr = 5e-3
sched_lr  = combine_scheds([0.3,0.7], cos_1cycle_anneal(lr/10., lr, lr/1e5))
sched_mom = combine_scheds([0.3,0.7], cos_1cycle_anneal(0.8, 0.7, 0.8))
cbsched = [ParamScheduler('lr', sched_lr), ParamScheduler('mom', sched_mom)]

In [ ]:
learn.fit(10, cbs=cbsched)

epoch,train_loss,train_accuracy_flat,valid_loss,valid_accuracy_flat,time
0,4.687608,0.291707,4.092570,0.326852,31:56
1,3.820686,0.346892,3.850637,0.339844,32:09
2,3.647742,0.356461,3.788184,0.343903,32:10
3,3.581744,0.360113,3.731966,0.346503,32:11
4,3.537887,0.363021,3.710345,0.349000,32:12
5,3.498959,0.366143,3.662489,0.353199,32:11
6,3.463220,0.369154,3.612004,0.357837,32:11
7,3.433527,0.371822,3.587005,0.359640,32:13
8,3.412087,0.373745,3.558273,0.363798,32:14
9,3.402041,0.374496,3.550977,0.364029,32:15


In [ ]:
torch.save(learn.model.state_dict(), path/'pretrained.pth')
pickle.dump(vocab, open(path/'vocab.pkl', 'wb'))